In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "custom_analysis_title"
      },
      "source": [
        "# Custom Market Analysis - Indian Stock Market Research System v1.0\n",
        "\n",
        "**Created:** January 2022  \n",
        "**Purpose:** Custom technical and fundamental analysis for Indian equity markets  \n",
        "**Market Focus:** NSE/BSE listed securities  "
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "setup_section"
      },
      "source": [
        "## 📦 Setup and Installation\n",
        "Install required packages for market data analysis"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "install_packages"
      },
      "outputs": [],
      "source": [
        "# Install required packages\n",
        "!pip install yfinance pandas numpy matplotlib seaborn plotly ta-lib-binary nsepy\n",
        "!pip install scipy scikit-learn warnings\n",
        "\n",
        "# Import libraries\n",
        "import yfinance as yf\n",
        "import pandas as pd\n",
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "import seaborn as sns\n",
        "import plotly.graph_objects as go\n",
        "import plotly.express as px\n",
        "from plotly.subplots import make_subplots\n",
        "import talib as ta\n",
        "from datetime import datetime, timedelta\n",
        "import warnings\n",
        "warnings.filterwarnings('ignore')\n",
        "\n",
        "# Set display options\n",
        "pd.set_option('display.max_columns', None)\n",
        "pd.set_option('display.width', None)\n",
        "plt.style.use('seaborn-v0_8')\n",
        "\n",
        "print(\"✅ All packages installed and imported successfully\")\n",
        "print(f\"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "indian_stocks_config"
      },
      "source": [
        "## 🇮🇳 Indian Stock Universe Configuration\n",
        "Define the universe of Indian stocks for analysis"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "stock_universe_setup"
      },
      "outputs": [],
      "source": [
        "# Indian Stock Universe (2022 Market Cap Leaders)\n",
        "INDIAN_STOCKS = {\n",
        "    'Large_Cap': {\n",
        "        'RELIANCE.NS': 'Reliance Industries',\n",
        "        'TCS.NS': 'Tata Consultancy Services',\n",
        "        'HDFCBANK.NS': 'HDFC Bank',\n",
        "        'INFY.NS': 'Infosys',\n",
        "        'HINDUNILVR.NS': 'Hindustan Unilever',\n",
        "        'ICICIBANK.NS': 'ICICI Bank',\n",
        "        'KOTAKBANK.NS': 'Kotak Mahindra Bank',\n",
        "        'SBIN.NS': 'State Bank of India',\n",
        "        'BHARTIARTL.NS': 'Bharti Airtel',\n",
        "        'ITC.NS': 'ITC Limited'\n",
        "    },\n",
        "    'IT_Sector': {\n",
        "        'TCS.NS': 'Tata Consultancy Services',\n",
        "        'INFY.NS': 'Infosys',\n",
        "        'WIPRO.NS': 'Wipro',\n",
        "        'HCLTECH.NS': 'HCL Technologies',\n",
        "        'TECHM.NS': 'Tech Mahindra'\n",
        "    },\n",
        "    'Banking': {\n",
        "        'HDFCBANK.NS': 'HDFC Bank',\n",
        "        'ICICIBANK.NS': 'ICICI Bank',\n",
        "        'KOTAKBANK.NS': 'Kotak Mahindra Bank',\n",
        "        'SBIN.NS': 'State Bank of India',\n",
        "        'AXISBANK.NS': 'Axis Bank'\n",
        "    },\n",
        "    'FMCG': {\n",
        "        'HINDUNILVR.NS': 'Hindustan Unilever',\n",
        "        'ITC.NS': 'ITC Limited',\n",
        "        'NESTLEIND.NS': 'Nestle India',\n",
        "        'BRITANNIA.NS': 'Britannia Industries',\n",
        "        'DABUR.NS': 'Dabur India'\n",
        "    }\n",
        "}\n",
        "\n",
        "# Indian Market Indices\n",
        "INDIAN_INDICES = {\n",
        "    '^NSEI': 'Nifty 50',\n",
        "    '^BSESN': 'BSE Sensex',\n",
        "    '^NSEBANK': 'Nifty Bank',\n",
        "    '^CNXIT': 'Nifty IT'\n",
        "}\n",
        "\n",
        "print(\"🇮🇳 Indian Stock Universe Configured\")\n",
        "print(f\"📊 Total Stocks: {sum(len(sector) for sector in INDIAN_STOCKS.values())}\")\n",
        "print(f\"📈 Market Indices: {len(INDIAN_INDICES)}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "data_fetching_section"
      },
      "source": [
        "## 📊 Data Fetching Functions\n",
        "Custom functions to fetch and process Indian market data"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "data_fetching_functions"
      },
      "outputs": [],
      "source": [
        "class IndianMarketDataFetcher:\n",
        "    \"\"\"Custom data fetcher for Indian stock market analysis\"\"\"\n",
        "    \n",
        "    def __init__(self):\n",
        "        self.data_cache = {}\n",
        "        \n",
        "    def fetch_stock_data(self, symbol, period='1y', interval='1d'):\n",
        "        \"\"\"Fetch stock data with error handling\"\"\"\n",
        "        try:\n",
        "            ticker = yf.Ticker(symbol)\n",
        "            data = ticker.history(period=period, interval=interval)\n",
        "            \n",
        "            if data.empty:\n",
        "                print(f\"⚠️ No data found for {symbol}\")\n",
        "                return None\n",
        "                \n",
        "            # Cache the data\n",
        "            self.data_cache[symbol] = data\n",
        "            return data\n",
        "            \n",
        "        except Exception as e:\n",
        "            print(f\"❌ Error fetching data for {symbol}: {str(e)}\")\n",
        "            return None\n",
        "    \n",
        "    def fetch_multiple_stocks(self, stock_dict, period='1y'):\n",
        "        \"\"\"Fetch data for multiple stocks\"\"\"\n",
        "        all_data = {}\n",
        "        \n",
        "        for symbol, name in stock_dict.items():\n",
        "            print(f\"📥 Fetching data for {name} ({symbol})\")\n",
        "            data = self.fetch_stock_data(symbol, period=period)\n",
        "            \n",
        "            if data is not None:\n",
        "                all_data[symbol] = {\n",
        "                    'name': name,\n",
        "                    'data': data\n",
        "                }\n",
        "        \n",
        "        return all_data\n",
        "    \n",
        "    def calculate_technical_indicators(self, data):\n",
        "        \"\"\"Calculate comprehensive technical indicators\"\"\"\n",
        "        if data is None or data.empty:\n",
        "            return None\n",
        "            \n",
        "        # Price data\n",
        "        high = data['High'].values\n",
        "        low = data['Low'].values\n",
        "        close = data['Close'].values\n",
        "        volume = data['Volume'].values\n",
        "        \n",
        "        indicators = data.copy()\n",
        "        \n",
        "        try:\n",
        "            # Moving Averages\n",
        "            indicators['SMA_20'] = ta.SMA(close, timeperiod=20)\n",
        "            indicators['SMA_50'] = ta.SMA(close, timeperiod=50)\n",
        "            indicators['EMA_12'] = ta.EMA(close, timeperiod=12)\n",
        "            indicators['EMA_26'] = ta.EMA(close, timeperiod=26)\n",
        "            \n",
        "            # RSI\n",
        "            indicators['RSI'] = ta.RSI(close, timeperiod=14)\n",
        "            \n",
        "            # MACD\n",
        "            macd, macd_signal, macd_hist = ta.MACD(close)\n",
        "            indicators['MACD'] = macd\n",
        "            indicators['MACD_Signal'] = macd_signal\n",
        "            indicators['MACD_Hist'] = macd_hist\n",
        "            \n",
        "            # Bollinger Bands\n",
        "            bb_upper, bb_middle, bb_lower = ta.BBANDS(close)\n",
        "            indicators['BB_Upper'] = bb_upper\n",
        "            indicators['BB_Middle'] = bb_middle\n",
        "            indicators['BB_Lower'] = bb_lower\n",
        "            \n",
        "            # Volume indicators\n",
        "            indicators['Volume_SMA'] = ta.SMA(volume, timeperiod=20)\n",
        "            \n",
        "            # ATR for volatility\n",
        "            indicators['ATR'] = ta.ATR(high, low, close, timeperiod=14)\n",
        "            \n",
        "            return indicators\n",
        "            \n",
        "        except Exception as e:\n",
        "            print(f\"⚠️ Error calculating indicators: {str(e)}\")\n",
        "            return data\n",
        "\n",
        "# Initialize the data fetcher\n",
        "fetcher = IndianMarketDataFetcher()\n",
        "print(\"✅ Indian Market Data Fetcher initialized\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "custom_analysis_section"
      },
      "source": [
        "## 🔍 Custom Analysis Functions\n",
        "Advanced analysis functions for Indian market insights"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "custom_analysis_functions"
      },
      "outputs": [],
      "source": [
        "class CustomMarketAnalyzer:\n",
        "    \"\"\"Custom analysis functions for Indian market research\"\"\"\n",
        "    \n",
        "    def __init__(self):\n",
        "        self.analysis_results = {}\n",
        "    \n",
        "    def sector_performance_analysis(self, sector_data):\n",
        "        \"\"\"Analyze sector performance and trends\"\"\"\n",
        "        performance_metrics = {}\n",
        "        \n",
        "        for symbol, stock_info in sector_data.items():\n",
        "            data = stock_info['data']\n",
        "            name = stock_info['name']\n",
        "            \n",
        "            if data is not None and not data.empty:\n",
        "                # Calculate returns\n",
        "                daily_returns = data['Close'].pct_change().dropna()\n",
        "                \n",
        "                # Performance metrics\n",
        "                total_return = (data['Close'].iloc[-1] / data['Close'].iloc[0] - 1) * 100\n",
        "                volatility = daily_returns.std() * np.sqrt(252) * 100\n",
        "                sharpe_ratio = (daily_returns.mean() * 252) / (daily_returns.std() * np.sqrt(252))\n",
        "                max_drawdown = self.calculate_max_drawdown(data['Close'])\n",
        "                \n",
        "                performance_metrics[symbol] = {\n",
        "                    'name': name,\n",
        "                    'total_return_pct': round(total_return, 2),\n",
        "                    'volatility_pct': round(volatility, 2),\n",
        "                    'sharpe_ratio': round(sharpe_ratio, 2),\n",
        "                    'max_drawdown_pct': round(max_drawdown, 2),\n",
        "                    'current_price': round(data['Close'].iloc[-1], 2)\n",
        "                }\n",
        "        \n",
        "        return pd.DataFrame(performance_metrics).T\n",
        "    \n",
        "    def calculate_max_drawdown(self, price_series):\n",
        "        \"\"\"Calculate maximum drawdown\"\"\"\n",
        "        cumulative = (1 + price_series.pct_change()).cumprod()\n",
        "        running_max = cumulative.expanding().max()\n",
        "        drawdown = (cumulative - running_max) / running_max\n",
        "        return drawdown.min() * 100\n",
        "    \n",
        "    def correlation_analysis(self, stock_data_dict):\n",
        "        \"\"\"Analyze correlations between stocks\"\"\"\n",
        "        price_data = {}\n",
        "        \n",
        "        for symbol, stock_info in stock_data_dict.items():\n",
        "            if stock_info['data'] is not None:\n",
        "                price_data[stock_info['name']] = stock_info['data']['Close']\n",
        "        \n",
        "        if len(price_data) > 1:\n",
        "            correlation_df = pd.DataFrame(price_data).corr()\n",
        "            return correlation_df\n",
        "        else:\n",
        "            return None\n",
        "    \n",
        "    def technical_signal_analysis(self, data_with_indicators):\n",
        "        \"\"\"Generate technical trading signals\"\"\"\n",
        "        signals = data_with_indicators.copy()\n",
        "        \n",
        "        # Initialize signal columns\n",
        "        signals['Signal'] = 0  # 0: Hold, 1: Buy, -1: Sell\n",
        "        signals['Signal_Strength'] = 0  # Signal strength (0-100)\n",
        "        \n",
        "        # RSI signals\n",
        "        signals.loc[signals['RSI'] < 30, 'RSI_Signal'] = 1  # Oversold - Buy\n",
        "        signals.loc[signals['RSI'] > 70, 'RSI_Signal'] = -1  # Overbought - Sell\n",
        "        signals['RSI_Signal'] = signals['RSI_Signal'].fillna(0)\n",
        "        \n",
        "        # MACD signals\n",
        "        signals['MACD_Signal_Cross'] = 0\n",
        "        macd_cross = (signals['MACD'] > signals['MACD_Signal']) & (signals['MACD'].shift(1) <= signals['MACD_Signal'].shift(1))\n",
        "        signals.loc[macd_cross, 'MACD_Signal_Cross'] = 1\n",
        "        \n",
        "        macd_cross_down = (signals['MACD'] < signals['MACD_Signal']) & (signals['MACD'].shift(1) >= signals['MACD_Signal'].shift(1))\n",
        "        signals.loc[macd_cross_down, 'MACD_Signal_Cross'] = -1\n",
        "        \n",
        "        # Moving Average signals\n",
        "        signals['MA_Signal'] = 0\n",
        "        signals.loc[signals['Close'] > signals['SMA_20'], 'MA_Signal'] = 1\n",
        "        signals.loc[signals['Close'] < signals['SMA_20'], 'MA_Signal'] = -1\n",
        "        \n",
        "        # Composite signal\n",
        "        signals['Composite_Signal'] = (signals['RSI_Signal'] + signals['MACD_Signal_Cross'] + signals['MA_Signal']) / 3\n",
        "        \n",
        "        return signals\n",
        "    \n",
        "    def market_regime_analysis(self, index_data):\n",
        "        \"\"\"Analyze market regime (Bull/Bear/Sideways)\"\"\"\n",
        "        if index_data is None or index_data.empty:\n",
        "            return None\n",
        "            \n",
        "        # Calculate moving averages for trend identification\n",
        "        index_data['SMA_50'] = index_data['Close'].rolling(window=50).mean()\n",
        "        index_data['SMA_200'] = index_data['Close'].rolling(window=200).mean()\n",
        "        \n",
        "        # Volatility (VIX proxy)\n",
        "        returns = index_data['Close'].pct_change()\n",
        "        volatility = returns.rolling(window=20).std() * np.sqrt(252) * 100\n",
        "        \n",
        "        # Current regime\n",
        "        current_close = index_data['Close'].iloc[-1]\n",
        "        current_sma50 = index_data['SMA_50'].iloc[-1]\n",
        "        current_sma200 = index_data['SMA_200'].iloc[-1]\n",
        "        current_volatility = volatility.iloc[-1]\n",
        "        \n",
        "        if current_close > current_sma50 > current_sma200:\n",
        "            regime = \"Bull Market\"\n",
        "        elif current_close < current_sma50 < current_sma200:\n",
        "            regime = \"Bear Market\"\n",
        "        else:\n",
        "            regime = \"Sideways Market\"\n",
        "        \n",
        "        return {\n",
        "            'regime': regime,\n",
        "            'current_price': round(current_close, 2),\n",
        "            'sma_50': round(current_sma50, 2),\n",
        "            'sma_200': round(current_sma200, 2),\n",
        "            'volatility': round(current_volatility, 2)\n",
        "        }\n",
        "\n",
        "# Initialize analyzer\n",
        "analyzer = CustomMarketAnalyzer()\n",
        "print(\"✅ Custom Market Analyzer initialized\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "live_analysis_section"
      },
      "source": [
        "## 📈 Live Market Analysis\n",
        "Perform real-time analysis on Indian stocks"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "live_market_analysis"
      },
      "outputs": [],
      "source": [
        "# Fetch current market data\n",
        "print(\"🔄 Starting Live Market Analysis...\")\n",
        "print(\"=\"*60)\n",
        "\n",
        "# 1. Fetch Large Cap stocks data\n",
        "print(\"\\n📊 Fetching Large Cap Stocks Data...\")\n",
        "large_cap_data = fetcher.fetch_multiple_stocks(INDIAN_STOCKS['Large_Cap'], period='6mo')\n",
        "\n",
        "# 2. Fetch Nifty 50 index data\n",
        "print(\"\\n📈 Fetching Nifty 50 Index Data...\")\n",
        "nifty_data = fetcher.fetch_stock_data('^NSEI', period='1y')\n",
        "\n",
        "print(f\"\\n✅ Data fetching completed at {datetime.now().strftime('%H:%M:%S')}\")\n",
        "print(f\"📊 Successfully fetched data for {len(large_cap_data)} stocks\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "sector_performance_analysis"
      },
      "outputs": [],
      "source": [
        "# Sector Performance Analysis\n",
        "print(\"🏭 SECTOR PERFORMANCE ANALYSIS\")\n",
        "print(\"=\"*50)\n",
        "\n",
        "if large_cap_data:\n",
        "    performance_df = analyzer.sector_performance_analysis(large_cap_data)\n",
        "    \n",
        "    # Sort by total return\n",
        "    performance_df_sorted = performance_df.sort_values('total_return_pct', ascending=False)\n",
        "    \n",
        "    print(\"\\n📊 Large Cap Performance Metrics (6 Months):\")\n",
        "    print(\"-\" * 80)\n",
        "    \n",
        "    for idx, (symbol, row) in enumerate(performance_df_sorted.iterrows(), 1):\n",
        "        print(f\"{idx:2d}. {row['name']:<25} | Return: {row['total_return_pct']:>6.1f}% | Vol: {row['volatility_pct']:>5.1f}% | Sharpe: {row['sharpe_ratio']:>5.2f}\")\n",
        "    \n",
        "    # Display full dataframe\n",
        "    print(\"\\n📋 Detailed Performance Table:\")\n",
        "    display(performance_df_sorted)\n",
        "else:\n",
        "    print(\"❌ No data available for analysis\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "technical_analysis"
      },
      "outputs": [],
      "source": [
        "# Technical Analysis for Top Performer\n",
        "print(\"🔧 TECHNICAL ANALYSIS\")\n",
        "print(\"=\"*40)\n",
        "\n",
        "if large_cap_data and not performance_df_sorted.empty:\n",
        "    # Get top performer\n",
        "    top_performer_symbol = performance_df_sorted.index[0]\n",
        "    top_performer_data = large_cap_data[top_performer_symbol]['data']\n",
        "    top_performer_name = large_cap_data[top_performer_symbol]['name']\n",
        "    \n",
        "    print(f\"\\n🎯 Analyzing: {top_performer_name} ({top_performer_symbol})\")\n",
        "    \n",
        "    # Calculate technical indicators\n",
        "    technical_data = fetcher.calculate_technical_indicators(top_performer_data)\n",
        "    \n",
        "    if technical_data is not None:\n",
        "        # Generate signals\n",
        "        signals = analyzer.technical_signal_analysis(technical_data)\n",
        "        \n",
        "        # Current values\n",
        "        current = signals.iloc[-1]\n",
        "        \n",
        "        print(f\"\\n📊 Current Technical Indicators:\")\n",
        "        print(f\"💰 Current Price: ₹{current['Close']:.2f}\")\n",
        "        print(f\"📈 SMA 20: ₹{current['SMA_20']:.2f}\")\n",
        "        print(f\"📊 RSI: {current['RSI']:.1f}\")\n",
        "        print(f\"🔄 MACD: {current['MACD']:.3f}\")\n",
        "        print(f\"📏 ATR: {current['ATR']:.2f}\")\n",
        "        \n",
        "        # Signal interpretation\n",
        "        composite_signal = current['Composite_Signal']\n",
        "        if composite_signal > 0.3:\n",
        "            signal_text = \"🟢 BUY Signal\"\n",
        "        elif composite_signal < -0.3:\n",
        "            signal_text = \"🔴 SELL Signal\"\n",
        "        else:\n",
        "            signal_text = \"🟡 HOLD Signal\"\n",
        "        \n",
        "        print(f\"\\n🎯 Composite Signal: {signal_text} (Score: {composite_signal:.2f})\")\n",
        "        \n",
        "        # Store for visualization\n",
        "        analysis_stock_data = technical_data\n",
        "        analysis_stock_name = top_performer_name\n",
        "        analysis_stock_symbol = top_performer_symbol\n",
        "    else:\n",
        "        print(\"❌ Could not calculate technical indicators\")\n",
        "else:\n",
        "    print(\"❌ No performance data available\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "visualization_section"
      },
      "source": [
        "## 📊 Advanced Visualizations\n",
        "Create comprehensive charts for market analysis"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "price_chart_visualization"
      },
      "outputs": [],
      "source": [
        "# Advanced Price Chart with Technical Indicators\n",
        "if 'analysis_stock_data' in locals():\n",
        "    print(f\"📈 Creating Advanced Chart for {analysis_stock_name}\")\n",
        "    \n",
        "    # Create subplots\n",
        "    fig = make_subplots(\n",
        "        rows=3, cols=1,\n",
        "        subplot_titles=(f'{analysis_stock_name} - Price & Moving Averages', 'RSI', 'MACD'),\n",
        "        vertical_spacing=0.05,\n",
        "        row_heights=[0.6, 0.2, 0.2]\n",
        "    )\n",
        "    \n",
        "    # Price and Moving Averages\n",
        "    fig.add_trace(\n",
        "        go.Candlestick(\n",
        "            x=analysis_stock_data.index,\n",
        "            open=analysis_stock_data['Open'],\n",
        "            high=analysis_stock_data['High'],\n",
        "            low=analysis_stock_data['Low'],\n",
        "            close=analysis_stock_data['Close'],\n",
        "            name='Price'\n",
        "        ),\n",
        "        row=1, col=1\n",
        "    )\n",
        "    \n",
        "    # Moving Averages\n",
        "    fig.add_trace(\n",
        "        go.Scatter(\n",
        "            x=analysis_stock_data.index,\n",
        "            y=analysis_stock_data['SMA_20'],\n",
        "            name='SMA 20',\n",
        "            line=dict(color='orange', width=2)\n",
        "        ),\n",
        "        row=1, col=1\n",
        "    )\n",
        "    \n",
        "    fig.add_trace(\n",
        "        go.Scatter(\n",
        "            x=analysis_stock_data.index,\n",
        "            y=analysis_stock_data['SMA_50'],\n",
        "            name='SMA 50',\n",
        "            line=dict(color='red', width=2)\n",
        "        ),\n",
        "        row=1, col=1\n",
        "    )\n",
        "    \n",
        "    # Bollinger Bands\n",
        "    fig.add_trace(\n",
        "        go.Scatter(\n",
        "            x=analysis_stock_data.index,\n",
        "            y=analysis_stock_data['BB_Upper'],\n",
        "            name='BB Upper',\n",
        "            line=dict(color='gray', width=1, dash='dash'),\n",
        "            showlegend=False\n",
        "        ),\n",
        "        row=1, col=1\n",
        "    )\n",
        "    \n",
        "    fig.add_trace(\n",
        "        go.Scatter(\n",


        not complted